# MISSION TO MARS

In this assignment, you will build a web application that scrapes various websites for data related to the Mission to Mars and displays the information in a single HTML page. The following outlines what you need to do.

In [37]:
#Dependencies
from bs4 import BeautifulSoup
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
import requests
import pymongo
import pandas as pd

## Step 1 - Scraping

### NASA Mars News

* Scrape the [NASA Mars News Site](https://mars.nasa.gov/news/) and collect the latest News Title and Paragraph Text. Assign the text to variables that you can reference later.


In [38]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [ ]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
#This finds only the top or newest news story
news_story = soup.find('li', 'slide')

#This scrapes the tile and teaser paragraph
for story in news_story:
    news_title = story.find('div','content_title').text
    news_para = story.find('div','article_teaser_body').text
    print(news_title)
    print(news_para)

### JPL Mars Space Images - Featured Image

* Visit the url for JPL Featured Space Image [here](https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars).

* Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url string to a variable called `featured_image_url`.

* Make sure to find the image url to the full size `.jpg` image.

* Make sure to save a complete url string for this image.


In [ ]:
# URL of page to be scraped
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars/'
browser.visit(url)

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
#This gets down to background-image:
background_image = soup.find('article')['style']
print(background_image)

In [ ]:
#Slice the string and add it to the rest of the path 
featured_image_url = "https://www.jpl.nasa.gov" + background_image[23:-3]
print(featured_image_url)

### Mars Weather

* Visit the Mars Weather twitter account [here](https://twitter.com/marswxreport?lang=en) and scrape the latest Mars weather tweet from the page. Save the tweet text for the weather report as a variable called `mars_weather`.


In [ ]:
# URL of page to be scraped
url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(url)

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
mars_weather = soup.find('p','TweetTextSize TweetTextSize--normal js-tweet-text tweet-text').text
print(mars_weather)

### Mars Facts

* Visit the Mars Facts webpage [here](http://space-facts.com/mars/) and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.

* Use Pandas to convert the data to a HTML table string.

In [ ]:
# URL of page to be scraped
url = 'http://space-facts.com/mars/'
browser.visit(url)

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
#read in any tables on the webpage
Mars_table = pd.read_html(url) 

In [ ]:
#Name the columns
mars_df = Mars_table[0]
mars_df.columns = ['descriptor','value']
#Set descriptor as index
mars_df.set_index('descriptor',inplace=True)
mars_df

In [ ]:
# Convert dataframe to HTML table
mars_html_table = mars_df.to_html()
mars_html_table

In [ ]:
#Strip unwanted newlines (\n)
mars_html_table.replace('\n','')

In [ ]:
mars_df.to_html('mars_table.html')

### Mars Hemispheres

* Visit the USGS Astrogeology site [here](https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars) to obtain high resolution images for each of Mar's hemispheres.

* You will need to click each of the links to the hemispheres in order to find the image url to the full resolution image.

* Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the hemisphere name. Use a Python dictionary to store the data using the keys `img_url` and `title`.

* Append the dictionary with the image url string and the hemisphere title to a list. This list will contain one dictionary for each hemisphere.

In [39]:
# URL of page to be scraped
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [42]:
products = soup.find('div', class_='collapsible results')
hemisphere = products.find_all('h3')

image_url_list = []
title_list =[]

for record in hemisphere:
    try:
        #Capture the title
        title_list.append(record)
        #Click on the link
        browser.click_link_by_partial_text('Enhanced')
        #find the Original Image link on the new page
        downloads = soup.find('div',class_='wide-image-wrapper')  #?? Why is this returning none? ??
        links = downloads.find_all('li')
        #original = links.find('li')[1]
        image_url_list.append(links)
    except ElementDoesNotExist:
        print("Scraping Complete")
    
titles_and_urls = zip(title_list, image_url_list)

print(titles_and_urls)

AttributeError: 'NoneType' object has no attribute 'find_all'

In [ ]:
print(title_list)

In [33]:
print(image_url_list)

[]


## Step 2 - MongoDB and Flask Application

Use MongoDB with Flask templating to create a new HTML page that displays all of the information that was scraped from the URLs above.

* Start by converting your Jupyter notebook into a Python script called `scrape_mars.py` with a function called `scrape` that will execute all of your scraping code from above and return one Python dictionary containing all of the scraped data.

* Next, create a route called `/scrape` that will import your `scrape_mars.py` script and call your `scrape` function.

  * Store the return value in Mongo as a Python dictionary.

* Create a root route `/` that will query your Mongo database and pass the mars data into an HTML template to display the data.

* Create a template HTML file called `index.html` that will take the mars data dictionary and display all of the data in the appropriate HTML elements. Use the following as a guide for what the final product should look like, but feel free to create your own design.


In [ ]:
'''# Display the MongoDB records created above
articles = db.articles.find()
for article in articles:
    print(article)'''

In [ ]:
########################